# Analyze accounts

In this step we look for interesting facts about the accounts.

In [1]:
import json
import os
import tarfile

import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import register_matplotlib_converters


register_matplotlib_converters()

%matplotlib inline

plt.style.use("seaborn")

## Load the data

In [2]:
data_directory = os.path.join("..", "..", "..", "data")

In [3]:
results_file_path = os.path.join(data_directory, "displacement_results.json")

if not os.path.exists(results_file_path):
    with tarfile.open(os.path.join(data_directory, "displacement_results.tar.xz"), "r:xz", encoding="utf-8") as compressed_file:
        compressed_file.extract("displacement_results.json", data_directory)

In [4]:
with open(results_file_path, "r", encoding="utf-8") as results_file:
    results = []
    line = results_file.readline().strip()
    while line != "":
        results.append(json.loads(line))
        line = results_file.readline().strip()

In [5]:
with open(os.path.join(data_directory, "displacement_bot_hash_to_cluster_id.json"), "r") as json_file:
    bot_hash_to_cluster_id = json.load(json_file)

In [6]:
with open(os.path.join(data_directory, "displacement_attacker_hash_to_cluster_id.json"), "r") as json_file:
    attacker_hash_to_cluster_id = json.load(json_file)

## Collect into data frames

In [7]:
Wei_to_KWei = 1e3
Wei_to_MWei = 1e6
Wei_to_GWei = 1e9
Wei_to_Ether = 1e18

In [8]:
attackers = []
bots = []
attack_blocks = []
attack_gas_prices = []
victim_senders = []
victim_receivers = []
victim_blocks = []
victim_gas_prices = []
costs = []
profits = []

for result in results:
    same_input = result["attacker_transaction"]["input"] == result["victim_transaction"]["input"]

    # the four entities should be different
    assert len(set([
        result["attacker_transaction"]["from"],
        result["attacker_transaction"]["to"],
        result["victim_transaction"]["from"],
        result["victim_transaction"]["to"]
    ])) == 4

    # the attacker should attack on an earlier block
    # or in the same block but with a lower index
    assert result["attacker_transaction"]["blockNumber"] < result["victim_transaction"]["blockNumber"]\
        or (result["attacker_transaction"]["blockNumber"] == result["victim_transaction"]["blockNumber"]\
        or result["attacker_transaction"]["transactionIndex"] < result["victim_transaction"]["transactionIndex"])

    # the attacker should not pay less for the gas
    assert result["attacker_transaction"]["gasPrice"] >= result["victim_transaction"]["gasPrice"]

    attackers.append(result["attacker_transaction"]["from"])
    bots.append(result["attacker_transaction"]["to"])
    attack_blocks.append(result["attacker_transaction"]["blockNumber"])
    attack_gas_prices.append(result["attacker_transaction"]["gasPrice"] / Wei_to_GWei)

    victim_senders.append(result["victim_transaction"]["from"])
    victim_receivers.append(result["victim_transaction"]["to"])
    victim_blocks.append(result["victim_transaction"]["blockNumber"])
    victim_gas_prices.append(result["victim_transaction"]["gasPrice"] / Wei_to_GWei)

    costs.append(result["cost_usd"])
    profits.append(result["profit_usd"])

# print some statistics
print("# attacks:", len(results))

# attacks: 2983


In [9]:
df_attacks = pd.DataFrame({
    "Attacker": attackers,
    "Bot": bots,
    "Attack Block": attack_blocks,
    "Attack Gas Price": attack_gas_prices,
    "Victim Sender": victim_senders,
    "Victim Receiver": victim_receivers,
    "Victim Block": victim_blocks,
    "Victim Gas Price": victim_gas_prices,
    "Cost": costs,
    "Profit": profits,
    "Bot Cluster ID": [bot_hash_to_cluster_id.get(bot) for bot in bots]  # will give None if bot clusters are not updated
})


df_attacks["Gas Price Delta"] = df_attacks["Attack Gas Price"] - df_attacks["Victim Gas Price"]
df_attacks["Block Delta"] = df_attacks["Victim Block"] - df_attacks["Attack Block"]

df_attacks.to_csv(os.path.join(data_directory, "displacement_attacks.csv"), index=False)

df_attacks

,Attacker,Bot,Attack Block,Attack Gas Price,Victim Sender,Victim Receiver,Victim Block,Victim Gas Price,Cost,Profit,Bot Cluster ID,Gas Price Delta,Block Delta
0,0xBa411E2E417794a327503C772684e85738a698b2,0x0E3358cA80A0e6Ef98C49747e2533b525d2a7A3b,1929171,21.612737,0x00bba23f50f25d550FCBf818C98d3C51dC612842,0x18a672E11D637fffADccc99B152F4895Da069601,1929189,20.0,0.052052,3.527918,1,1.612737e+00,18
1,0xC9D81352fBdb0294b091e51d774A0652ef776D99,0x85C5c26DC2aF5546341Fc1988B9d178148b4838B,6058392,5.000000,0x98Ac18627bF2205A816EEE7fbC919a7DB83A4908,0x12459C951127e0c374FF9105DdA097662A027093,6058393,1.6,0.563183,106.584745,2,3.400000e+00,1
2,0xC9D81352fBdb0294b091e51d774A0652ef776D99,0x85C5c26DC2aF5546341Fc1988B9d178148b4838B,6058392,5.000000,0xB036AE3d480A5EFf5A22ce3F9Db669E326e5f937,0x12459C951127e0c374FF9105DdA097662A027093,6058395,1.0,0.563183,106.584745,2,4.000000e+00,3
3,0x125D657d5Cd16Bf4864a2850D3F2541d9a0F3b50,0x3aCd4658a768d7C52BDEda2e8Bd8b7947A70C78B,6465900,8.000000,0x406c6cA639c42F038dDa19dF4897945006AB9aAB,0x4fC996F4AC70427c1402db2B72ff135fb049d559,6465900,2.3,0.094619,2.185742,3,5.700000e+00,0
4,0x541fB7476aab8d6E19904461932Cc9C9cf0F8d7C,0xC798D5Ed657adeF1cA7bc6090c590319dfFE8979,6013474,33.000000,0x7133497069c9eBF3bd099cA00d962c9e687565Aa,0x3f1603602e870BD4c6953333B35eC988eEde4f9C,6013474,11.0,3.074271,9.225125,4,2.200000e+01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2978,0x36451DC166Ee52Af314B64f760fD2e38E5906F50,0xe920B3F96Cb2bF1Ee9ee8921837de08273724242,10776884,494.000000,0x4D953115678b15CE0B0396bCF95Db68003f86FB5,0xF2a997f56373bC99fA6829098c78F10088255e77,10776884,494.0,26.294506,0.685300,11,9.999894e-10,0
2979,0x36451DC166Ee52Af314B64f760fD2e38E5906F50,0xe920B3F96Cb2bF1Ee9ee8921837de08273724242,10777139,487.000000,0x4D953115678b15CE0B0396bCF95Db68003f86FB5,0xF2a997f56373bC99fA6829098c78F10088255e77,10777139,487.0,24.755120,0.675589,11,9.999894e-10,0
2980,0x36451DC166Ee52Af314B64f760fD2e38E5906F50,0xe920B3F96Cb2bF1Ee9ee8921837de08273724242,10651210,367.000000,0x0385b3F162a0e001b60Ecb84D3CB06199d78f666,0x103675510a219bd84CE91d1bcb82Ca194D665a09,10651210,367.0,79.193415,3214.561097,11,9.999894e-10,0
2981,0x36451DC166Ee52Af314B64f760fD2e38E5906F50,0xe920B3F96Cb2bF1Ee9ee8921837de08273724242,10777415,479.000000,0x0385b3F162a0e001b60Ecb84D3CB06199d78f666,0x103675510a219bd84CE91d1bcb82Ca194D665a09,10777415,479.0,45.819417,671.730876,11,9.999894e-10,0


In [10]:
df_bots = df_attacks.groupby("Bot").agg({
        "Bot": "count",
        "Attacker": lambda attacker: attacker.nunique(),
        "Attack Block": "min",
        "Cost": "sum",
        "Profit": "sum",
    })\
    .rename(columns={
        "Bot": "Attacks",
        "Attacker": "Attackers",
        "Attack Block": "First Block",
    })\
    .sort_values(by="Attacks", ascending=False)\
    .reset_index()

df_bots["Cluster ID"] = df_bots["Bot"].map(lambda bot: bot_hash_to_cluster_id.get(bot))  # will give None if bot clusters are not updated

df_bots

,Bot,Attacks,Attackers,First Block,Cost,Profit,Cluster ID
0,0xe920B3F96Cb2bF1Ee9ee8921837de08273724242,1975,1,10567155,37189.293857,3.693931e+06,11
1,0x40DDE6092a77eC2d00eB4fa14f0c5d92d835d673,503,8,10016367,1255.820435,3.839096e+05,7
2,0x9a6de75b5ED999fBDc3FF71FC56562742531b35a,270,1,10470541,3223.016277,4.583376e+05,11
3,0x581cdeA80ebE14A8240eE11ADe3dd8973010967A,64,12,9983067,47.842063,1.933282e+04,7
4,0xf261fA1a1F649f9045fdDc157A503D5a832c0897,59,5,9932570,84.902692,4.392912e+03,7
5,0xC798D5Ed657adeF1cA7bc6090c590319dfFE8979,37,1,5646898,65.775476,3.809711e+03,4
6,0xF12E1AAec6F1f3fee011A7BE5AE53633286b3bBB,29,8,11025616,372.641675,5.130599e+03,16
7,0x103675510a219bd84CE91d1bcb82Ca194D665a09,6,3,10324864,22.146237,2.174687e+03,12
8,0xC1d78F0822E0ca30adfF484311A77ec7cA3dBBA9,6,1,8743918,2.613970,2.338382e+01,6
9,0x9799b475dEc92Bd99bbdD943013325C36157f383,5,5,10414279,254.506364,6.788882e+03,17


In [11]:
df_bots.to_csv(os.path.join(data_directory, "displacement_bots.csv"), index=False)

## Save latex tables for the publication

In [12]:
df_attacks[[
        "Cost",
        "Profit",
        "Gas Price Delta",
        "Block Delta",
    ]].describe().style.format("{:.2f}")

,Cost,Profit,Gas Price Delta,Block Delta
count,2983.00,2983.00,2983.00,2983.00
mean,14.28,1537.99,0.43,0.78
std,18.25,7162.80,2.65,2.37
min,0.01,0.00,0.00,0.00
25%,4.36,1.14,0.00,0.00
50%,9.48,158.53,0.00,0.00
75%,16.64,851.04,0.00,0.00
max,311.69,223150.01,52.90,19.00


In [13]:
latex = df_attacks[[
        "Cost",
        "Profit",
        "Gas Price Delta",
        "Block Delta",
    ]].describe().to_latex(index=True, formatters={
        "Cost": "{:,.2f}".format,
        "Profit": "{:,.2f}".format,
        "Gas Price Delta": "{:,.2f}".format,
        "Block Delta": "{:,.2f}".format,
    })

with open(os.path.join(data_directory, "displacement_attack_stats.tex"), "w") as latex_file:
    latex_file.write(latex)

## Other stats

### Attack variable correlations

In [14]:
df_attacks[[
        "Cost",
        "Profit",
        "Gas Price Delta",
        "Block Delta",
    ]].corr().style.format("{:.2f}").background_gradient(cmap='coolwarm')

,Cost,Profit,Gas Price Delta,Block Delta
Cost,1.00,0.14,-0.05,0.04
Profit,0.14,1.00,-0.03,-0.00
Gas Price Delta,-0.05,-0.03,1.00,0.24
Block Delta,0.04,-0.00,0.24,1.00
